<div id="colab_button">
  <h1>Quick tour</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindbox/blob/main/docs/docs/how-to-guides/prepare-app.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

## Introduction
______________________________

In this how-to guide, we are going to walk you through how you can create a basic application image with Docker and deploy it on a Confidential VM with custom security policies using BlindBox, allowing you to offer our users stringent privacy guarantees.

But before we go any further, if you want to follow along with this how-to guide, you'lld need to make sure you have the following pre-requisites installed or set-up.

### Pre-requisites

To follow along with this guide and deploy your own BlindBox, you will need:

+ An **Azure account** and **credits**. 

	> **Important**: This how-to guide deploys our "hello world" app on a DCasv5/ECasv5 series confidential VM with Azure. If you follow along with the Quick Tour, you will be charged according to your usage as detailed by Azure. You can get a detailed pricing quote [here](https://learn.microsoft.com/en-us/azure/confidential-computing/confidential-vm-overview).

+ Have Docker installed in your environment. Here's the [Docker installation guide](https://docs.docker.com/desktop/install/linux-install/).

#### Installing BlindBox and Terraform

Let's dive in! We'll first need to install the **[blindbox](https://pypi.org/project/blindbox/) PyPi package** and **`terraform`**, which will facilitate the deployment of our BlindBox.

In [ ]:
# install blindbox
!pip install blindbox

# Download terraform
!wget -O- https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg

# Add terraform to trusted apt packages
!echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list

# Install terraform
!sudo apt update && sudo apt install terraform

#### Azure CLI and login

We will also need to download the **Azure CLI tool** and **log in** to our Azure account.

In [ ]:
# Install azure-cli
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# Install confcom extension for azure-cli
!az extension add -n confcom

# Log in using azure-cli
!az login

# Set the subscription azure-cli will use by default.
!az account set --subscription "Microsoft Azure Sponsorship"

### Our application code

Now that this is done, let's take a look at our application code that will be launched on our confidential VM by BlindBox.

```python
import uvicorn
from fastapi import FastAPI

# initialize our API app object
app = FastAPI()

# use fastapi decorator to turn our Hello World function into API endpoints on our uvicorn server
@app.get("/hello")
def hello() -> str:
    return "Hello World"

if __name__ == "__main__":
    # deploy our server on port 80
    uvicorn.run(app, host="0.0.0.0", port=80)

```

You can view this file by downloading our github repository and taking a look at the `server.py` file in the `examples/hello-world` folder.

In [ ]:
!git clone https://github.com/mithril-security/blindbox
!cd examples/hello-world

This program does the following:

+ Creates the API app object that "configures" the uvicorn server by providing handlers for specific endpoints

+ Creates a `hello` endpoint on our `uvicorn server` which users will be able to query on our BlindBox address once packaged and deployed.

+ Deploys our API on a python ASGI server (an asynchronous web server) on port 80.

>Note it is important that your application is configured to run on port 80 as BlindBox will forward and receive traffic to the application on this port!

### Packaging our application in a Docker image

Next, we need to package our application in a Docker image which will then be wrapped in our BlindBox image with custom security policies and deployed on our Confidential VM.

Let's take a look at our Dockerfile we used to package this Hello World application:

```docker
FROM python:3.10-alpine

# download any requisites
RUN pip install \
        # provides handlers for specific endpoints on our server
        fastapi==0.95.0 \
        # gives support for multipart requests
        python-multipart==0.0.6 \
        # uvicorn is a python ASGI server (an asynchronous web server)
        uvicorn==0.21.1 \
        # fastapi uses this library to define response/requests formats and to serialize/deserialize the response/requests
        pydantic==1.10.7 \ 

# copy our application code across to our container
COPY server.py /

# signal that application will run on port 80
EXPOSE 80

# set command on launch to launch our application code
CMD ["python", "/server.py"]
```

Again, this file can be viewed in the `examples` folder in the official BlindBox GitHub repository.

There are no complex requirements for our Docker image, but it is recommended to `EXPOSE` port 80 to signal that our application will be running on port 80 within our BlindBox.

### Building our application

Next, we need to build our Hello World application image before we can deploy it with BlindBox.

In [ ]:
# build application assigning it the tag "hello-world"
!docker build -t hello-world .

### Preparing the BlindBox

Now we are ready to initialize our BlindBox by running the `blindbox init` command within the BlindBox `examples/hello-world` folder. This step will automatically create two template configuration files, the `blindbox.yml` and the `blindbox.tf` file.

+ The `blindbox.yml` file is used to set up custom security policies. It currently supports the following modifiable options:
	+ `platform` to specify the TEE type you want to use (we currently only support `amd-sev`)
	+ `ip-rules` to whitelist the `ip addresses` of the users allowed to query the BlindBox application. 
+ The `blindbox.tf` file contains all the configuration options for the Confidential VM which your BlindBox image will be deployed on. 

> The blindbox.yml config file should be modified before building your BlindBox for any changes to take effect. 

> The blindbox.tf config file should be modified before deploying your BlindBox for any changes to take effect.

In [ ]:
# initialize the blindbox with AMD-SEV TEE
!blindbox --platform azure-sev init 

We specify that we want to use the AMD SEV TEE platform with Azure using the `platform` option with `azure-sev` as our argument.

> Note this is the only platform currently supported- `aws-nitro` is **coming soon**.

We should now be able to see the `blindbox.yml` and `blindbox.tf` files in our `examples/hello-world` folder.

#### Building the BlindBox image

Next we are ready to build our BlindBox image which is a wrapper around your application image. It implements the custom security policies specified in the `blindbox.yml` file.

We do this using `blindbox build` and specifying the application image using the `--source-image` option.

In [ ]:
# build the application image with the tag "hello-world-blindbox"
!blindbox build -t hello-world-blindbox --source-image hello-world

### Deploying our application on a Confidental VM

We next use the `deploy` command to deploy our `blindbox image` (which contains the inner `application image`) on an `Azure confidential VM`. 

In [ ]:
# deploy our image inside Confidential VM using BlindBox
!blindbox deploy hello-world-blindbox

### Querying the enclave

Now that the deployment is done, authorised users can query the model within the enclave using the `requests` module within the `blindbox` library we installed earlier on.

In [ ]:
# import requests module
from blindbox  import requests

# we query our application on our hello endpoint
res = requests.post(url="http://localhost:8080/enclave/hello")

# display result
print(res)

As you can see, by querying the `hello` endpoint of our `hello world application`, we get back the expected result: `hello world`!

## Conclusion
_______________________
 
In this tutorial, we've seen how we can:
+ Create a BlindBox-compatible application
+ Package and build our application
+ Create and build our BlindBox
+ Deploy our Blindox on a Confidential VM